<a href="https://colab.research.google.com/github/ImitatedSky/Python_AI/blob/main/news_sentiment_analysis_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

# data download from https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news/data

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/dataset') #切換該目錄

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding
from keras.layers import SpatialDropout1D, Conv1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
header = ["Sentiment", "Contents"]
df = pd.read_csv('sentiment_data.csv', encoding="ISO-8859-1", names=header )


In [6]:
df


,Sentiment,Contents
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [7]:
random_index = np.random.permutation(df.index)
df = df.loc[random_index]

sentiment_mapping = {
    "positive": 2,
    "neutral": 1,
    "negative": 0
}
df['Sentiment'] = df['Sentiment'].map(sentiment_mapping)

In [8]:
df

,Sentiment,Contents
3416,1,These module products will be available for tr...
3305,1,The order is a follow-on to an 11 mln euro ( $...
2992,2,"Jobs will not be transferred from Finland , ho..."
3955,1,To see a slide show of all the newest product ...
3128,1,"Sponda is a property investment company , spec..."
...,...,...
2619,1,The cranes would be installed onboard two frei...
101,2,Circulation revenue has increased by 5 % in Fi...
911,2,"The planned facility , estimated to cost aroun..."
3825,1,The divested activities had net sales of EUR 1...


In [9]:
# Create  dataframe
# data = df.filter(['Contents'])
# Convert the dataframe to a numpy array
dataset = df.values
# 將所有的95%放下去測試
training_data_len = int(np.ceil( len(dataset) * .95 ))

print(f"總共資料筆數{len(dataset)} , 訓練資料筆數{training_data_len}")

總共資料筆數4846 , 訓練資料筆數4604


In [10]:
train_data = dataset[0:int(training_data_len), :]

x_train = train_data[:,1]
y_train = train_data[:,0]

test_data = dataset[training_data_len : , :]

x_test = test_data[:,1]
y_test = test_data[:,0]


In [11]:
pad_type = trunc_type = 'pre'
max_review_length = 36

In [12]:
# 文本向量化
from keras.preprocessing.text import Tokenizer

# 初始化一個Tokenizer
tokenizer = Tokenizer()

# 適應訓練及數列
tokenizer.fit_on_texts(x_train)
vocab_size = len(tokenizer.word_index)+1
print(vocab_size)

# 將訓練集 測試集 轉為整數序列
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

9883


In [13]:
import tensorflow as tf
from keras.utils import to_categorical

# 再使用pad_sequences函数填充序列
x_train = pad_sequences(x_train, maxlen = max_review_length , padding=pad_type, truncating=trunc_type, value=0, dtype='int32')
x_test = pad_sequences(x_test, maxlen = max_review_length , padding=pad_type, truncating=trunc_type, value=0, dtype='int32')

x_train = tf.convert_to_tensor(x_train, dtype=tf.int32)
y_train = to_categorical(y_train)

x_test = tf.convert_to_tensor(x_test, dtype=tf.int32)
y_test = to_categorical(y_test)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPool1D , Dropout, Embedding

model = Sequential()

# 添加嵌入層，將整数序列轉為嵌入向量
model.add(Embedding( input_dim = vocab_size , output_dim = 100 , input_length = max_review_length ))

model.add(SpatialDropout1D( 0.2 ))

model.add(Conv1D(196, 3, activation='relu'))
model.add(Conv1D(196, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(196, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.summary()

# 编译模型
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=20)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 36, 100)           988300    
                                                                 
 spatial_dropout1d (Spatial  (None, 36, 100)           0         
 Dropout1D)                                                      
                                                                 
 conv1d (Conv1D)             (None, 34, 196)           58996     
                                                                 
 conv1d_1 (Conv1D)           (None, 32, 196)           115444    
                                                                 
 global_max_pooling1d (Glob  (None, 196)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 196)               3

In [15]:
predictions = model.predict(x_test)
loss, acc = model.evaluate(x_test, y_test)
print(f"accuracy：{acc}")
print(f"loss：{loss}")

8/8 [==============================] - 0s 10ms/step - loss: 1.4729 - accuracy: 0.7562
accuracy：0.7561983466148376
loss：1.472923755645752


In [20]:
_input = input("sentiment analysis contents：")
_input = tokenizer.texts_to_sequences([_input])
_input = pad_sequences(_input, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0, dtype='int32')
_input = tf.convert_to_tensor(_input, dtype=tf.int32)

#
predictions = model.predict(_input)

#
sentiment_label = np.argmax(predictions)  # 返回最大的

#
sentiment_mapping = {
    2: "positive",
    1: "neutral",
    0: "negative"
}

predicted_sentiment = sentiment_mapping[sentiment_label]
print("prediction ：", predicted_sentiment)
predictions


sentiment analysis contents：Airbus workers stage rare protest over job cuts
1/1 [==============================] - 0s 33ms/step
prediction ： negative


array([[0.87662566, 0.08438092, 0.0389934 ]], dtype=float32)